<a href="https://colab.research.google.com/github/EricEricEricJin/ECE539-Group-Project/blob/master/pred_series_mlp_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras

import os
import datetime

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load dataset

DIR_NAME = "/content/drive/My Drive/Colab Notebooks/ECE539_Project/"

def get_series_labels():
  series = []
  labels = []
  for dirname, _, filenames in os.walk(DIR_NAME + "pred_series"):
    if ("singing" in dirname or "speech" in dirname):
      for filename in filenames:
        for row in np.load(os.path.join(dirname, filename)):
          series.append(row)
          labels.append([0, 1] if "singing" in dirname else [1, 0])
  return series, labels

series, labels = get_series_labels()
series_ds = tf.data.Dataset.from_tensor_slices((series, labels))
print("Number of data", len(series_ds))

for _ in series_ds.take(1):
  input_size = len(_[0])
print("Input size", input_size)

Number of data 152
Input size 50


In [ ]:
# Train test splittion
batch_size = 8
series_ds_batched = series_ds.batch(batch_size)
train_ds, val_ds = keras.utils.split_dataset(series_ds_batched, left_size=0.8)
# train_ds = train_ds.batch(batch_size)
# val_ds = val_ds.batch(batch_size)

In [ ]:
# Develop a three-layer MLP with 50 inputs and 2 outputs
model = keras.models.Sequential()
model.add(keras.layers.Dense(50, input_dim=input_size, activation="ReLU"))
model.add(keras.layers.Dense(60, activation="ReLU"))
model.add(keras.layers.Dense(50, activation="ReLU"))
model.add(keras.layers.Dense(2, activation="softmax"))
model.compile(loss='binary_crossentropy', optimizer='adam',
            metrics=['accuracy'])

EPOCHS = 50

checkpoint_filepath = f"/content/drive/My Drive/Colab Notebooks/ECE539_Project/pred_series_mlp_weights_{str(datetime.date.today())}"
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

earlystop_callback = keras.callbacks.EarlyStopping(verbose=1, patience=30)


history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[earlystop_callback, checkpoint_callback],
)

Epoch 1/50
16/16 [==============================] - 3s 79ms/step - loss: 0.8121 - accuracy: 0.5000 - val_loss: 1.1832 - val_accuracy: 0.0000e+00
Epoch 2/50
16/16 [==============================] - 1s 65ms/step - loss: 0.3507 - accuracy: 0.8770 - val_loss: 0.6139 - val_accuracy: 0.7000
Epoch 3/50
16/16 [==============================] - 1s 66ms/step - loss: 0.2677 - accuracy: 0.9508 - val_loss: 0.3918 - val_accuracy: 0.8333
Epoch 4/50
16/16 [==============================] - 1s 46ms/step - loss: 0.2062 - accuracy: 0.9508 - val_loss: 0.2917 - val_accuracy: 0.8667
Epoch 5/50
16/16 [==============================] - 1s 50ms/step - loss: 0.1657 - accuracy: 0.9590 - val_loss: 0.2379 - val_accuracy: 0.9333
Epoch 6/50
16/16 [==============================] - 0s 5ms/step - loss: 0.1424 - accuracy: 0.9672 - val_loss: 0.2205 - val_accuracy: 0.9333
Epoch 7/50
16/16 [==============================] - 0s 6ms/step - loss: 0.1272 - accuracy: 0.9754 - val_loss: 0.2236 - val_accuracy: 0.9333
Epoch 8/50
